## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

from astropy import units
from astropy.cosmology import FlatLambdaCDM, z_at_value

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import simulation
import simtrace
import match_halo
import dataset_comp

In [ ]:
import importlib
importlib.reload(simulation)
importlib.reload(simtrace)
importlib.reload(match_halo)
importlib.reload(dataset_comp)

# Evolution Histories of the M31 and MW satellites around the time of fall-in


---

### plain-LCDM-LR

Choose the simulations, and define the M31 and the MW at redshift $z=0$:

In [ ]:
env_path = os.path.abspath(os.path.join('..', 'test_tracing_inj'))

sim = simulation.Simulation("V1_LR_fix", env_path=env_path)

snap_z0 = 127
m31_id_z0 = (1, 0)
mw_id_z0 = (2, 0)

---

## Tracing

Set the range of snapshots to be traced:

In [ ]:
snap_start = 100
snap_stop = 128
snap_ids = np.arange(snap_start, snap_stop)

In [ ]:
# If the simulations are not already linked:
matcher = match_halo.SnapshotMatcher(n_link_ref=20, n_matches=1)
mtree = simtrace.MergerTree(sim, matcher=matcher, branching="BackwardBranching")
mtree.build_tree(snap_start, snap_stop)

In [ ]:
# Trace subhalos and get the M31 and the MW Subhalo objects:
sub_dict = sim.trace_subhalos(snap_start, snap_stop)
m31 = sub_dict[snap_z0][
    sim.get_snapshot(snap_z0).index_of_halo(m31_id_z0[0], m31_id_z0[1])
]
mw = sub_dict[snap_z0][
    sim.get_snapshot(snap_z0).index_of_halo(mw_id_z0[0], mw_id_z0[1])
]

In [ ]:
fallin_m31, fallin_mw = simtrace.get_fallin_times_lg(sim, m31, mw, snap_start, snap_stop)

## Check Fallin Times

First, simply check fall-in array shapes:

In [ ]:
shapes_match = [(fallin_m31[snap_id].size == sim.get_snapshot(snap_id).get_subhalo_number()) and
                (fallin_mw[snap_id].size == sim.get_snapshot(snap_id).get_subhalo_number()) 
                for snap_id in snap_ids]
print(np.all(shapes_match))

The fall-in snapshot arrays should have non-NaN values for exactly those elements that represent satellites:

In [ ]:
snap_check = [101, 105, 111, 120, 127]

In [ ]:
for snap_id in snap_check:
    mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
        sim.get_snapshot(snap_id), m31, mw, sat_r=300, comov=True
    )
    mask_sats2, mask_isol_2 = dataset_comp.split_satellites_by_distance_old(
        sim.get_snapshot(snap_id), m31_id_z0, mw_id_z0, max_dist_sat=300
    )

    print(np.all(mask_m31 == ~np.isnan(fallin_m31[snap_id])))
    print(np.all(mask_mw == ~np.isnan(fallin_mw[snap_id])))

### Inspect coordinates at fall-in

Coordinates at fall-in should, trivially, be at a circle of radius 300 ckpc around the central. That is, except for those satellites that form as satellite, or which already were satellites at `min(snap_ids)`.

In [ ]:
mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
    sim.get_snapshot(snap_z0), m31, mw, sat_r=300, comov=True
)
mask_sats2, mask_isol_2 = dataset_comp.split_satellites_by_distance_old(
    sim.get_snapshot(snap_z0), m31_id_z0, mw_id_z0, max_dist_sat=300
)

In [ ]:
m31_sats_z0 = sub_dict[snap_z0][mask_m31]

In [ ]:
dist_m31 = {sid: np.linalg.norm(d, axis=1) * units.cm.to(units.kpc)
            for sid, d in m31.distance_to_self(snap_ids).items()}

In [ ]:
print(dataset_comp.get_subhalos_at_fallin(m31_sats_z0, fallin_m31, dist_m31))

In [ ]:
print(fallin_m31[snap_z0][mask_m31])

## Plot radial evolution with fall-in

In [ ]:
redshift = {sid: z for sid, z in zip(snap_ids, sim.get_attribute("Redshift", "Header", snap_ids))}

In [ ]:
m31_sats = sub_dict[snap_z0][mask_m31]
d_at_fallin, sid_at_fallin = dataset_comp.get_subhalos_at_fallin(m31_sats, fallin_m31, dist_m31)
data = {"M31_Satellites": {
    "Snap_id": [np.array(sat.get_indices())[1] for sat in m31_sats],
    "Redshift": [np.array([redshift[sid] for _, sid in zip(*sat.get_indices())])
                 for sat in m31_sats],
    "Distance": [dataset_comp.subhalo_dataset_from_dict(sat, dist_m31)[0] for sat in m31_sats],
    "D_at_fallin": [d for d in d_at_fallin],
    "Z_at_fallin": [sim.get_attribute("Redshift", "Header", sid)[0] for sid in sid_at_fallin]
}}

In [ ]:
fig, ax = plt.subplots()

ax.invert_xaxis()

# Plot 300ckpc:
a = sim.get_attribute('Time', 'Header', snap_ids)
z = sim.get_attribute('Redshift', 'Header', snap_ids)
ax.plot(z, a * 300, c='gray', linestyle='--')

i1=20; i2=30
for r, z in zip(data["M31_Satellites"]["Distance"][i1:i2],
                data["M31_Satellites"]["Redshift"][i1:i2]):
    ax.plot(z, r)
    ax.scatter(z, r, s=10)
    
for rf, zf in zip(data["M31_Satellites"]["D_at_fallin"][i1:i2],
                  data["M31_Satellites"]["Z_at_fallin"][i1:i2]):
    ax.scatter(zf, rf, c='black', s=15)

# for subhalo in sub_dict[snap_z0][mask_m31]:
#     d, sub_snaps = dataset_comp.get_subhalo_from_data_dict(subhalo, dist_m31)
#     z = sim.get_redshifts(sub_snaps)
#     ax.plot(z,d)